## Import Library

In [1]:
import os
import cv2
import pandas as pd
import numpy as np
import math
import timm
import yaml
import random
import torch
import torch.nn as nn
import albumentations
import albumentations.pytorch

from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import _LRScheduler, CosineAnnealingLR, StepLR
from sklearn.model_selection import StratifiedKFold, train_test_split
from tqdm.auto import tqdm
from tqdm.notebook import tqdm
from easydict import EasyDict
from torchvision import transforms
from PIL import Image

In [2]:
patience = 10
counter = 0

num_epochs = 50
accumulation_steps = 2
batch_size = 64
train_log_interval = 100

LEARNING_RATE = 0.0001 
lr_decay_step = 5

## Bring csv & Delete noise data

In [3]:
filename = 'OpenData_PotOpenTabletIdntfc20220412.xls'
df = pd.read_excel(filename, engine='openpyxl')

In [4]:
## delete 구강정 데이터
index_delete1 = df[df['품목일련번호']==200605327].index
index_delete2 = df[df['품목일련번호']==200605328].index
index_delete3 = df[df['품목일련번호']==200605329].index
index_delete4 = df[df['품목일련번호']==200605330].index
index_delete5 = df[df['품목일련번호']==200605331].index
index_delete6 = df[df['품목일련번호']==200606263].index

## delete 반원형 데이터
index_delete7 = df[df['품목일련번호']==197800388].index
index_delete8 = df[df['품목일련번호']==199906868].index
index_delete9 = df[df['품목일련번호']==197900378].index

In [5]:
df = df.drop(index_delete1)
df = df.drop(index_delete2)
df = df.drop(index_delete3)
df = df.drop(index_delete4)
df = df.drop(index_delete5)
df = df.drop(index_delete6)
df = df.drop(index_delete7)
df = df.drop(index_delete8)
df = df.drop(index_delete9)

In [7]:
is_text_nan = []
for front, back in df.iloc[:, [6, 7]].values:
    # nan: 0, text: 1
    if type(front) is float and type(back) is float:
        is_text_nan.append(0)
    else:
        is_text_nan.append(1)

In [8]:
df.insert(29, 'is_text_nan', is_text_nan)

In [9]:
num_classes = len(set(is_text_nan))

## Create CustomDataset

In [10]:
class CustomDataset(Dataset):
    def __init__(self, df, transform=None):
        super().__init__()
        self.df = df.reset_index()
        self.image_id = self.df['품목일련번호']
        self.labels = self.df['is_text_nan']
        self.transform = transform
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self,idx):
        image_id = self.image_id[idx]
        label = self.labels[idx]
        image_path = f'/opt/ml/data_handling/data/{image_id}.jpg'
        image = Image.open(image_path)
        if self.transform:
            image = self.transform(image)
        
        return image, label

## Split Train/Val

In [11]:
image_num = df['품목일련번호']
label = df['is_text_nan']

# https://teddylee777.github.io/scikit-learn/train-test-split
x_train, x_valid, y_train, y_valid = train_test_split(image_num, label, test_size=0.2, stratify=label, random_state=22)

In [12]:
## https://mizykk.tistory.com/131

train_zip = zip(x_train, y_train)
train_df = pd.DataFrame(train_zip)
train_df.columns = ['품목일련번호','is_text_nan']

val_zip = zip(x_valid, y_valid)
val_df = pd.DataFrame(val_zip)
val_df.columns = ['품목일련번호','is_text_nan']

In [13]:
train_df.groupby('is_text_nan').count()

,품목일련번호
is_text_nan,
0,147
1,19343


In [14]:
train_df.groupby('is_text_nan').count().sum()

품목일련번호    19490
dtype: int64

In [15]:
val_df.groupby('is_text_nan').count()

,품목일련번호
is_text_nan,
0,37
1,4836


In [16]:
val_df.groupby('is_text_nan').count().sum()

품목일련번호    4873
dtype: int64

In [17]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

train_dataset = CustomDataset(train_df, transform=transform)
val_dataset = CustomDataset(val_df, transform=transform)

In [18]:
image, label = next(iter(train_dataset))
image, label

(tensor([[[0.7077, 0.7077, 0.7077,  ..., 0.7077, 0.7077, 0.7077],
          [0.7077, 0.7077, 0.7077,  ..., 0.7077, 0.7077, 0.7077],
          [0.7077, 0.7077, 0.7077,  ..., 0.7077, 0.7077, 0.7077],
          ...,
          [0.7077, 0.7077, 0.7077,  ..., 0.7077, 0.7077, 0.6906],
          [0.7077, 0.7077, 0.7077,  ..., 0.7077, 0.7077, 0.6906],
          [0.6734, 0.6906, 0.6906,  ..., 0.6734, 0.6734, 0.6734]],
 
         [[1.3256, 1.3256, 1.3256,  ..., 1.3256, 1.3256, 1.3256],
          [1.3256, 1.3256, 1.3256,  ..., 1.3256, 1.3256, 1.3256],
          [1.3256, 1.3256, 1.3256,  ..., 1.3256, 1.3256, 1.3256],
          ...,
          [1.3256, 1.3256, 1.3256,  ..., 1.3256, 1.3256, 1.3081],
          [1.3256, 1.3256, 1.3256,  ..., 1.3256, 1.3256, 1.3081],
          [1.2906, 1.3081, 1.3081,  ..., 1.2906, 1.2906, 1.2906]],
 
         [[2.2914, 2.2914, 2.2914,  ..., 2.2914, 2.2914, 2.2914],
          [2.2914, 2.2914, 2.2914,  ..., 2.2914, 2.2914, 2.2914],
          [2.2914, 2.2914, 2.2914,  ...,

In [19]:
image, label = next(iter(val_dataset))
image, label

(tensor([[[1.1529, 1.1529, 1.1529,  ..., 1.1529, 1.1529, 1.1529],
          [1.1529, 1.1529, 1.1529,  ..., 1.1529, 1.1529, 1.1529],
          [1.1529, 1.1529, 1.1529,  ..., 1.1529, 1.1529, 1.1529],
          ...,
          [1.1529, 1.1529, 1.1529,  ..., 1.1529, 1.1529, 1.1529],
          [1.1529, 1.1529, 1.1529,  ..., 1.1529, 1.1529, 1.1529],
          [1.1529, 1.1529, 1.1529,  ..., 1.1529, 1.1529, 1.1529]],
 
         [[1.5707, 1.5707, 1.5707,  ..., 1.5707, 1.5707, 1.5707],
          [1.5707, 1.5707, 1.5707,  ..., 1.5707, 1.5707, 1.5707],
          [1.5707, 1.5707, 1.5707,  ..., 1.5707, 1.5707, 1.5707],
          ...,
          [1.5707, 1.5707, 1.5707,  ..., 1.5707, 1.5707, 1.5707],
          [1.5707, 1.5707, 1.5707,  ..., 1.5707, 1.5707, 1.5707],
          [1.5707, 1.5707, 1.5707,  ..., 1.5707, 1.5707, 1.5707]],
 
         [[2.1171, 2.1171, 2.1171,  ..., 2.1171, 2.1171, 2.1171],
          [2.1171, 2.1171, 2.1171,  ..., 2.1171, 2.1171, 2.1171],
          [2.1171, 2.1171, 2.1171,  ...,

In [20]:
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_dataset, shuffle=False, batch_size=batch_size)

## Pretrained Model

In [21]:
model_name = 'resnet50'
model = timm.create_model(model_name, pretrained=True, num_classes=num_classes)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
model.to(device) 

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act3): ReLU(inplace=True)
      (downsample): Sequen

In [22]:
criterion = torch.nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE) 
scheduler = StepLR(optimizer, lr_decay_step, gamma=0.5)

In [23]:
import wandb
wandb.init(project="final-project", entity="medic", name=f"KM_{model_name}_text_nan")

name = f'{model_name}_type_and_shape'
os.makedirs(os.path.join(os.getcwd(), 'results', name), exist_ok=True)

counter = 0
best_val_acc = 0
best_val_loss = np.inf

for epoch in range(num_epochs):
    # train loop
    model.train()
    loss_value = 0
    matches = 0
    for idx, train_batch in tqdm(enumerate(train_loader)):
        inputs, labels = train_batch
        inputs = inputs.to(device)
        labels = labels.to(device)

        outs = model(inputs)
        preds = torch.argmax(outs, dim=-1)
        loss = criterion(outs, labels)

        loss.backward()
        
        # -- Gradient Accumulation
        if (idx+1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        loss_value += loss.item()
        matches += (preds == labels).sum().item()
        if (idx + 1) % train_log_interval == 0:
            train_loss = loss_value / train_log_interval
            train_acc = matches / batch_size / train_log_interval
            current_lr = scheduler.get_last_lr()
            print(
                f"Epoch[{epoch}/{num_epochs}]({idx + 1}/{len(train_loader)}) || "
                f"training loss {train_loss:4.4} || training accuracy {train_acc:4.2%} || lr {current_lr}"
            )

            loss_value = 0
            matches = 0
            
    scheduler.step()

    # val loop
    with torch.no_grad():
        print("Calculating validation results...")
        model.eval()
        val_loss_items = []
        val_acc_items = []
        label_accuracy, total_label = [0]*num_classes, [0]*num_classes
        for val_batch in val_loader:
            inputs, labels = val_batch
            inputs = inputs.to(device)
            labels = labels.to(device)

            outs = model(inputs)
            preds = torch.argmax(outs, dim=-1)

            loss_item = criterion(outs, labels).item()
            acc_item = (labels == preds).sum().item()
            val_loss_items.append(loss_item)
            val_acc_items.append(acc_item)
            
            ## label별 accuracy
            for i in range(len(labels)):
                total_label[int(labels[i])] += 1
                if labels[i] == preds[i]:
                    label_accuracy[int(labels[i])] += 1
            

        val_loss = np.sum(val_loss_items) / len(val_loader)
        val_acc = np.sum(val_acc_items) / len(val_df)
        
        # Callback1: validation accuracy가 향상될수록 모델을 저장합니다.
        if val_loss < best_val_loss:
            best_val_loss = val_loss
        if val_acc > best_val_acc:
            print("New best model for val accuracy! saving the model..")
            torch.save(model.state_dict(), f"results/{name}/best.ckpt")
            best_val_acc = val_acc
            counter = 0
        else:
            counter += 1
        # Callback2: patience 횟수 동안 성능 향상이 없을 경우 학습을 종료시킵니다.
        if counter > patience:
            print("Early Stopping...")
            break
        
        ## 파이썬 배열 나눗셈 https://bearwoong.tistory.com/60
        accuracy_by_label = np.array(label_accuracy)/np.array(total_label)
        print(f"accuracy by label: {accuracy_by_label}")
        
        print(
            f"[Val] acc : {val_acc:4.2%}, loss: {val_loss:4.2} || "
            f"best acc : {best_val_acc:4.2%}, best loss: {best_val_loss:4.2}"
        )
        
        print(
            f"nan: {accuracy_by_label[0]}\n"
            f"text: {accuracy_by_label[1]}\n"
        )

    wandb.log({
        "train_loss": train_loss,
        "train_accuracy": train_acc,
        "valid_loss": val_loss,
        "valid_accuracy": val_acc,
        "best_loss": best_val_loss,
        "best_accuracy": best_val_acc,
        "nan": accuracy_by_label[0],
        "text": accuracy_by_label[1]
    })

wandb: Currently logged in as: seoulsky_field (medic). Use `wandb login --relogin` to force relogin


Epoch[0/50](100/305) || training loss 0.05971 || training accuracy 98.38% || lr [0.0001]
Epoch[0/50](200/305) || training loss 0.01239 || training accuracy 99.58% || lr [0.0001]
New best model for val accuracy! saving the model..
accuracy by label: [0.89189189 0.99979322]
[Val] acc : 99.90%, loss: 0.0052 || best acc : 99.90%, best loss: 0.0052
nan: 0.8918918918918919
text: 0.999793217535153



Epoch[1/50](100/305) || training loss 0.008419 || training accuracy 99.73% || lr [0.0001]
Epoch[1/50](200/305) || training loss 0.003749 || training accuracy 99.92% || lr [0.0001]
Epoch[1/50](300/305) || training loss 0.004406 || training accuracy 99.86% || lr [0.0001]

Calculating validation results...
accuracy by label: [0.7027027  0.99979322]
[Val] acc : 99.75%, loss: 0.0048 || best acc : 99.90%, best loss: 0.0048
nan: 0.7027027027027027
text: 0.999793217535153



Epoch[2/50](100/305) || training loss 0.003737 || training accuracy 99.88% || lr [0.0001]
Epoch[2/50](200/305) || training loss 0.007684 || training accuracy 99.66% || lr [0.0001]
Epoch[2/50](300/305) || training loss 0.005171 || training accuracy 99.83% || lr [0.0001]

Calculating validation results...
accuracy by label: [0.78378378 0.99979322]
[Val] acc : 99.82%, loss: 0.01 || best acc : 99.90%, best loss: 0.0048
nan: 0.7837837837837838
text: 0.999793217535153



Epoch[3/50](100/305) || training loss 0.004662 || training accuracy 99.91% || lr [0.0001]
Epoch[3/50](200/305) || training loss 0.003932 || training accuracy 99.89% || lr [0.0001]
Epoch[3/50](300/305) || training loss 0.001807 || training accuracy 99.97% || lr [0.0001]

Calculating validation results...
accuracy by label: [0.89189189 0.99937965]
[Val] acc : 99.86%, loss: 0.0042 || best acc : 99.90%, best loss: 0.0042
nan: 0.8918918918918919
text: 0.999379652605459



...

Epoch[13/50](100/305) || training loss 2.902e-05 || training accuracy 100.00% || lr [2.5e-05]
Epoch[13/50](200/305) || training loss 2.29e-05 || training accuracy 100.00% || lr [2.5e-05]
Epoch[13/50](300/305) || training loss 4.968e-05 || training accuracy 100.00% || lr [2.5e-05]

Calculating validation results...
accuracy by label: [0.94594595 0.99958644]
[Val] acc : 99.92%, loss: 0.0037 || best acc : 99.94%, best loss: 0.0022
nan: 0.9459459459459459
text: 0.9995864350703061



Epoch[14/50](100/305) || training loss 4.913e-05 || training accuracy 100.00% || lr [2.5e-05]
Epoch[14/50](200/305) || training loss 2.232e-05 || training accuracy 100.00% || lr [2.5e-05]
Epoch[14/50](300/305) || training loss 3.668e-05 || training accuracy 100.00% || lr [2.5e-05]

Calculating validation results...
accuracy by label: [0.94594595 0.99937965]
[Val] acc : 99.90%, loss: 0.0037 || best acc : 99.94%, best loss: 0.0022
nan: 0.9459459459459459
text: 0.999379652605459



Epoch[15/50](100/305) || training loss 3.182e-05 || training accuracy 100.00% || lr [1.25e-05]
Epoch[15/50](200/305) || training loss 2.409e-05 || training accuracy 100.00% || lr [1.25e-05]
Epoch[15/50](300/305) || training loss 1.919e-05 || training accuracy 100.00% || lr [1.25e-05]

Calculating validation results...
accuracy by label: [0.94594595 0.99896609]
[Val] acc : 99.86%, loss: 0.0047 || best acc : 99.94%, best loss: 0.0022
nan: 0.9459459459459459
text: 0.9989660876757651



Epoch[16/50](100/305) || training loss 1.814e-05 || training accuracy 100.00% || lr [1.25e-05]


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [24]:
class CustomTestDataset(Dataset):
    def __init__(self, df, transform=None):
        super().__init__()
        self.df = df.reset_index()
        self.image_id = self.df['품목일련번호']
        self.transform = transform
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self,idx):
        image_id = self.image_id[idx]
        image_path = f'/opt/ml/final-project-level3-cv-16/test_data/{image_id}'
        image = Image.open(image_path)
        if self.transform:
            image = self.transform(image)
        
        return image

In [25]:
test_batch_size = 1

In [26]:
image_dir = './test_data'
test_list = os.listdir(image_dir)
test_list = sorted(test_list)

In [27]:
test_df = pd.DataFrame(test_list)
test_df.columns = ['품목일련번호']

In [28]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

test_dataset = CustomTestDataset(test_df, transform=transform)

In [29]:
image = next(iter(test_dataset))
image

tensor([[[1.2043, 1.1700, 1.1358,  ..., 0.2967, 0.2282, 0.3138],
         [1.2214, 1.1015, 1.1872,  ..., 0.1768, 0.1597, 0.2453],
         [1.1358, 1.1187, 1.1358,  ..., 0.2282, 0.2796, 0.2796],
         ...,
         [0.6392, 0.6221, 0.5878,  ..., 0.6734, 0.5707, 0.6221],
         [0.4508, 0.4508, 0.5022,  ..., 0.7077, 0.7933, 0.7591],
         [0.5022, 0.5364, 0.5022,  ..., 0.6906, 0.7933, 0.7933]],

        [[1.4307, 1.3782, 1.3431,  ..., 0.3452, 0.2402, 0.3803],
         [1.4482, 1.3256, 1.4132,  ..., 0.2052, 0.1877, 0.2927],
         [1.3431, 1.3081, 1.3431,  ..., 0.2927, 0.3277, 0.3277],
         ...,
         [0.7129, 0.6779, 0.6429,  ..., 0.7654, 0.6429, 0.6954],
         [0.4853, 0.4853, 0.5378,  ..., 0.8179, 0.9230, 0.8880],
         [0.5553, 0.5728, 0.5203,  ..., 0.8004, 0.9230, 0.9230]],

        [[1.5245, 1.5071, 1.4548,  ..., 0.4439, 0.3219, 0.4439],
         [1.5768, 1.4374, 1.5420,  ..., 0.3045, 0.2696, 0.3742],
         [1.4722, 1.4374, 1.4722,  ..., 0.3742, 0.4091, 0.

In [30]:
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=test_batch_size)

In [31]:
model = timm.create_model('resnet50', pretrained=True, num_classes=num_classes)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
model.to(device) 

PATH = f"./results/{name}/best.ckpt"
model.load_state_dict(torch.load(PATH, map_location=device))

<All keys matched successfully>

In [32]:
answer= ['nan', 'text']

In [33]:
# val loop
with torch.no_grad():
    print("Calculating validation results...")
    model.eval()
    val_loss_items = []
    val_acc_items = []
    for val_batch in test_loader:
        inputs = val_batch
        inputs = inputs.to(device)

        outs = model(inputs)
        preds = torch.argmax(outs, dim=-1)
        
        print(answer[int(preds)])     

Calculating validation results...
text
text
text
nan
text
nan
text
nan
nan
nan
text
nan
nan
text
nan
text
nan
text
nan
nan
nan
text
nan
nan
nan
nan
nan
nan
text
text
text
text
text
